In [0]:
%pip install -Uq "protobuf>=4.21.6"  qdrant-client tiktoken "arize-phoenix[evals,embeddings]" "openai>=1" openinference-instrumentation-langchain litellm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-feature-engineering 0.7.0 requires protobuf<5,>=3.12.0, but you have protobuf 5.29.1 which is incompatible.
mlflow-skinny 2.15.1 requires importlib-metadata!=4.7.0,<8,>=3.7.0, but you have importlib-metadata 8.4.0 which is incompatible.
tensorboard-plugin-profile 2.17.0 requires protobuf<5.0.0dev,>=3.19.6, but you have protobuf 5.29.1 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:

# %pip uninstall -y langchain langchain-core langchain-openai langchain-community langchain-groq langchain-text-splitters openinference-instrumentation-langchain

%pip install --no-cache-dir "langchain>=0.1.0,<0.4.0"
%pip install --no-cache-dir "langchain-core>=0.1.0,<0.4.0"
%pip install --no-cache-dir "langchain-openai>=0.0.2"
%pip install --no-cache-dir "langchain-community>=0.0.10"
%pip install --no-cache-dir "langchain-groq>=0.2.0"
%pip install --no-cache-dir "langchain-text-splitters>=0.0.1"
%pip install --no-cache-dir "openinference-instrumentation-langchain>=0.1.29"

dbutils.library.restartPython()

# Verify installations (run after restart)
%pip freeze | grep langchain


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/50.7 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/410.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 61.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.41
    Not uninstalling langchain-core at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-7e5cbdd1-3a45-49c3-887c-69c57584de4a
    Can't uninstall 'langchain-core'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are ins

In [0]:
dbutils.library.restartPython()

In [0]:

from langchain_core.messages.ai import InputTokenDetails
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from openinference.instrumentation.langchain import LangChainInstrumentor

# Print package versions to verify
import pkg_resources
packages = [
    'langchain',
    'langchain-community',
    'langchain-core',
    'langchain-groq',
    'langchain-openai',
    'langchain-text-splitters',
    'openinference-instrumentation-langchain'
]

for package in packages:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"{package}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package}: Not found")

langchain: 0.3.11
langchain-community: 0.3.11
langchain-core: 0.3.24
langchain-groq: 0.2.1
langchain-openai: 0.2.12
langchain-text-splitters: 0.3.2
openinference-instrumentation-langchain: 0.1.29


In [0]:

import os
import json
import tempfile
from getpass import getpass
from urllib.request import urlretrieve

import nest_asyncio
import numpy as np
import pandas as pd
from sklearn.metrics import ndcg_score
from langchain.callbacks import StdOutCallbackHandler

from langchain.chains import RetrievalQA, LLMChain
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings

from openinference.instrumentation.langchain import LangChainInstrumentor


import phoenix as px
from phoenix.otel import register

In [0]:
px.close_app()

In [0]:
# Configuration and Initialization
nest_asyncio.apply()
pd.set_option("display.max_colwidth", None)

# Configure Groq API Key using dbutils
try:
    groq_api_key = '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'
    os.environ["GROQ_API_KEY"] = groq_api_key
    os.environ["PHOENIX_PROJECT_NAME"] = "Phoenix_Capabilities_Testing"
    os.environ["AZURE_API_KEY"] = "38a6b22e0e4f43828877d844399faf4d"
    os.environ["AZURE_API_BASE"] = "https://ai-abhinavkatiyarai793972137108.openai.azure.com" 
    os.environ["AZURE_API_VERSION"] = "2024-08-01-preview"

except Exception as e:
    print("Error accessing Groq API key from secrets. Please add it to the 'llm-keys' scope with key 'groq-api-key'")
    raise e


# Configure Azure OpenAI Embeddings
embeddings = OpenAIEmbeddings(
    deployment="text-embedding-ada-002",  # Your Azure OpenAI deployment name
    model="text-embedding-ada-002",      # OpenAI model name
    api_type="azure",
    api_base="https://abhin-m2ifqtz5-eastus2.openai.azure.com",  # Azure OpenAI endpoint
    api_version="2023-05-15",    # Azure API version for embeddings
    api_key="4f46e8f30eac4a3abedeb41c04b7ab52"  # Azure OpenAI API key
)


/root/.ipykernel/75992/command-4402110491910851-2227220095:20: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-7e5cbdd1-3a45-49c3-887c-69c57584de4a/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:271: UserWarning: WARNING! api_type is not default parameter.
                    api_type was transferred to model_kwargs.
                    Please confirm that api_type is what you intended.
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-7e5cbdd1-3a45-49c3-887c-69c57584de4a/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:271: UserWarning: WARNING! api_bas

In [0]:
# Configure embeddings using SentenceTransformers
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)





def load_documents(directory_path):
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  
            file_path = os.path.join(directory_path, filename)
            loader = TextLoader(file_path)
            documents.extend(loader.load())
    return documents

documents = load_documents('/Workspace/Users/abhinav.katiyar@spaceinventive.com/data/')

# Create text splitter for smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)

# Process documents into chunks
all_chunks = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)
    valid_chunks = [chunk for chunk in chunks if len(chunk) > 100]
    all_chunks.extend(valid_chunks)

/root/.ipykernel/75992/command-4402110491910852-2342295847:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
2024-12-11 05:28:13.650251: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-11 05:28:13.777212: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-11 05:28:13.901957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/databricks/python/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [0]:
from langchain_openai import AzureOpenAIEmbeddings
import os

# Configure Azure OpenAI embeddings
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://abhin-m2ifqtz5-eastus2.openai.azure.com",
    deployment="text-embedding-ada-002",  # Changed from azure_deployment to deployment
    api_key="4f46e8f30eac4a3abedeb41c04b7ab52",
    api_version="2023-05-15"
)

def load_documents(directory_path):
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  
            file_path = os.path.join(directory_path, filename)
            loader = TextLoader(file_path)
            documents.extend(loader.load())
    return documents

documents = load_documents('/Workspace/Users/abhinav.katiyar@spaceinventive.com/data/')

# Create text splitter for smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)

# Process documents into chunks
texts = text_splitter.split_documents(documents)  # Changed to split_documents instead of manual splitting

# Build vector store
qdrant = Qdrant.from_documents(
    texts,
    embeddings,
    location=":memory:",
    collection_name="my_documents",
)

# Configure retriever
retriever = qdrant.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 2,
        "fetch_k": 3
    }
)

2024/12/11 05:28:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.12/site-packages/mlflow/tracing/utils/__init__.py:53: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel
"


[Trace(request_id=tr-3141e605f5f04ab6a7380fd7477617ca), Trace(request_id=tr-842751f40b8449d9af07f74b74701c93)]

In [0]:

# Build vector store
qdrant = Qdrant.from_documents(
    texts,
    embeddings,
    location=":memory:",
    collection_name="my_documents",
)

# Configure retriever
retriever = qdrant.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 2,
        "fetch_k": 3
    }
)

# Configure Groq for question generation
question_llm = ChatGroq(
    model_name="mixtral-8x7b-32768",
    temperature=0.0,
    max_tokens=512,
    streaming=False
)

# Configure Groq for QA
qa_llm = ChatGroq(
    model_name="mixtral-8x7b-32768",
    temperature=0.1,
    max_tokens=1024,
    streaming=False
)

[Trace(request_id=tr-5832e30c82944ab880cdb11d8a31be8f), Trace(request_id=tr-0e5663eb6c094ac68501ca641ce98192)]

In [0]:

# Question Generation Template
# generate_questions_template = """Create exactly 3 questions based on this text. Only return a JSON object.

# TEXT TO ANALYZE:
# {text}

# RESPONSE FORMAT:
# {{
# "question_1": "Write your first question here",
# "question_2": "Write your second question here",
# "question_3": "Write your third question here"
# }}

# IMPORTANT: Only return the JSON object, no additional text."""

generate_questions_template = """\
Context information is below.

---------------------
{text}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
3 questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."

Output the questions in JSON format with the keys question_1, question_2, question_3.
"""

# QA Template
qa_prompt_template = """Answer the following question based on the given context. Be concise.

Context: {context}

Question: {question}

Answer:"""

# Create chains
question_chain = LLMChain(
    llm=question_llm,
    prompt=PromptTemplate(
        template=generate_questions_template,
        input_variables=["text"]
    )
)

qa_chain = RetrievalQA.from_chain_type(
    llm=qa_llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=qa_prompt_template,
            input_variables=["context", "question"]
        )
    }
)

/root/.ipykernel/75992/command-4402110491910854-926698858:45: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  question_chain = LLMChain(


In [0]:
session = px.active_session()
print(session)

In [0]:
import phoenix as px
print(px.__version__)

7.0.1


In [0]:
session = px.Session(endpoint="http://localhost:6006") 

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-159938735837198>, line 1
----> 1 session = px.Session(endpoint="http://localhost:6006")

TypeError: Can't instantiate abstract class Session without an implementation for abstract methods 'active', 'end'

In [0]:
tracer_provider = register()
LangChainInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

# Launch Phoenix
session = px.launch_app()
print(f"Phoenix UI available at: {session.url}")

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Phoenix_Capabilities_Testing
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

🌍 To view the Phoenix app in your browser, visit https://adb-182495964472138.18.azuredatabricks.net/driver-proxy/o/182495964472138/1205-082119-n66oclg8/6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
Phoenix UI available at: https://adb-182495964472138.18.azuredatabricks.net/driver-proxy/o/182495964472138/1205-082119-n66oclg8/6006/


In [0]:
question = "What information is available in the context?"

response = qa_chain({"query": question})
print(response['query'])
print(response['result'])

2024/12/11 05:29:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.12/site-packages/mlflow/utils/autologging_utils/safety.py:558: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead."


What information is available in the context?
The context provides information about the variation in the burden of proof for establishing causation in hybrid human-AI decision chains across different jurisdictions. It also mentions a business case study related to GlobalTech's market position, but no specific details or information about this case are given.


Trace(request_id=tr-317ef45af04047c5bef3755ca10fd948)

In [0]:
spans_df = px.Client().get_spans_dataframe(project_name="Phoenix_Capabilities_Testing")
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents","attributes.llm.output_messages"]].head(2)



,name,span_kind,attributes.input.value,attributes.retrieval.documents,attributes.llm.output_messages
context.span_id,,,,,
ebed2652caff587f,VectorStoreRetriever,RETRIEVER,What information is available in the context?,"[{'document.content': '. Pursuant to subsection (c)(2), operators must maintain comprehensive documentation of decision paths for any autonomous actions affecting Category A systems (as defined in Appendix II-B).', 'document.metadata': {'_id': '7521ad8bff3940d18234c357bc85e5c2', '_collection_name': 'my_documents'}}, {'document.content': '2. Philosophical questions about AI agency and responsibility 3. Regulatory compliance across multiple jurisdictions 4. Impact on market dynamics and competitive positioning', 'document.metadata': {'_id': 'e45ea86acad148c1b62af39cec666af5', '_collection_name': 'my_documents'}}]",None
525656ab0a6010de,ChatGroq,LLM,"{""messages"": [[{""lc"": 1, ""type"": ""constructor"", ""id"": [""langchain"", ""schema"", ""messages"", ""HumanMessage""], ""kwargs"": {""content"": ""Answer the following question based on the given context. Be concise.\n\nContext: . Pursuant to subsection (c)(2), operators must maintain comprehensive documentation of decision paths for any autonomous actions affecting Category A systems (as defined in Appendix II-B).\n\n2. Philosophical questions about AI agency and responsibility\n3. Regulatory compliance across multiple jurisdictions\n4. Impact on market dynamics and competitive positioning\n\nQuestion: What information is available in the context?\n\nAnswer:"", ""type"": ""human""}}]]}",None,"[{'message.role': 'assistant', 'message.content': 'The context provides information about a regulation related to autonomous actions affecting certain systems (Category A systems) in the context of AI. It mentions that operators must maintain comprehensive documentation of decision paths for these autonomous actions. However, it does not provide information about philosophical questions, regulatory compliance across multiple jurisdictions, or impact on market dynamics and competitive positioning.'}]"


In [0]:
sampled_chunks = pd.DataFrame({"text": all_chunks})
sample_size = min(4, len(sampled_chunks))
sampled_chunks = sampled_chunks.sample(n=sample_size, random_state=42)

def clean_and_parse_response(response_text):
    """Clean and parse the response text into valid JSON."""
    try:
        cleaned = response_text.strip()
        start = cleaned.find('{')
        end = cleaned.rfind('}')
        
        if start != -1 and end != -1:
            cleaned = cleaned[start:end+1]
        
        result = json.loads(cleaned)
        
        required_keys = ['question_1', 'question_2', 'question_3']
        if not all(key in result for key in required_keys):
            raise ValueError("Missing required question keys")
            
        return result
    except Exception as e:
        print(f"Parsing error: {str(e)}")
        raise

# Generate questions
questions = []
for idx, row in sampled_chunks.iterrows():
    try:
        chunk_text = row['text'][:500]  # Limit chunk size
        
        response = question_chain.invoke({"text": chunk_text})
        parsed = clean_and_parse_response(response['text'])
        
        questions.append({
            "text": row['text'],  # Keep the original text chunk
            "question_1": parsed['question_1'],
            "question_2": parsed['question_2'],
            "question_3": parsed['question_3']
        })
        
    except Exception as e:
        print(f"Failed to process chunk {idx + 1}: {str(e)}")
        continue

# Create questions dataframe
questions_df = pd.DataFrame(questions)


# LLM_Generate by Phoneix (provide dby phoenix to generate questions)
import json

from phoenix.evals import OpenAIModel, llm_generate


def output_parser(response: str, index: int):
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        return {"__error__": str(e)}


questions_df = llm_generate(
    dataframe=document_chunks_df,
    template=generate_questions_template,
    model=OpenAIModel(model="gpt-3.5-turbo"),
    output_parser=output_parser,
    concurrency=20,
)

In [0]:
import json
from phoenix.evals import LiteLLMModel, llm_generate

sampled_chunks = pd.DataFrame({"text": all_chunks})
sample_size = min(5, len(sampled_chunks))
sampled_chunks = sampled_chunks.sample(n=sample_size, random_state=42)


def output_parser(response: str, index: int):
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        return {"__error__": str(e)}

questions_df = llm_generate(
    dataframe=sampled_chunks,
    template=generate_questions_template,
    model=LiteLLMModel(model="azure/gpt-35-turbo"),
    output_parser=output_parser,
    concurrency=20,
)

llm_generate |          | 0/5 (0.0%) | ⏳ 00:00<? | ?it/s

[Trace(request_id=tr-91af121fc4514f738ac0413903d84452), Trace(request_id=tr-f20d4e3640d64860beb80c64bc5f2d48), Trace(request_id=tr-a8c35e010280458caa94e7379c0a9bf3), Trace(request_id=tr-7972dc426f8d4cacb649bea6bd317cc5), Trace(request_id=tr-e730c68a0fcc4637bcc7ce448026192a)]

In [0]:
questions_df

,question_1,question_2,question_3
9,What is the participatory anthropic principle proposed by Wheeler?,"According to the context information, what is the implication of consciousness being primary?",Does objective reality exist independent of observation? Explain your answer with reference to the context information provided.
25,What is the 'substantial factor' test and which courts have adopted it?,What is the traditional causation standard for algorithmic influence assessment and how has it been modified?,What is the Model Autonomous Systems Code and how does it relate to the 'substantial factor' test?
8,What is the perspective proposed by some theorists regarding consciousness and reality?,How does the perspective of consciousness being fundamental to reality challenge materialist and dualist frameworks?,What is the difference between the emergent and fundamental perspectives of consciousness in relation to reality?
21,What is the requirement for operators regarding documentation of decision paths for autonomous actions affecting Category A systems?,What is the definition of Category A systems as per Appendix II-B?,What is the subsection that specifies the requirement for operators to maintain comprehensive documentation of decision paths for autonomous actions affecting Category A systems?
0,What is the Standard Model's conceptualization of quantum chromodynamics (QCD)?,What are strong interactions in the context of quantum chromodynamics (QCD)?,What is the role of quarks and gluons in quantum chromodynamics (QCD)?


In [0]:
# Construct a dataframe of the questions and the document chunks
questions_with_document_chunk_df = pd.concat([questions_df, sampled_chunks], axis=1)
questions_with_document_chunk_df = questions_with_document_chunk_df.melt(
    id_vars=["text"], value_name="question"
).drop("variable", axis=1)
# If the above step was interrupted, there might be questions missing. Let's run this to clean up the dataframe.
questions_with_document_chunk_df = questions_with_document_chunk_df[
    questions_with_document_chunk_df["question"].notnull()
]

In [0]:
questions_with_document_chunk_df

,text,question
0,"Consider the implications: if consciousness is primary, does objective reality exist independent of observation? This recalls Wheeler's participatory anthropic principle, suggesting that observers are necessary for the actualization of potential states",What is the participatory anthropic principle proposed by Wheeler?
1,". Some courts have adopted the ""substantial factor"" test outlined in the Model Autonomous Systems Code, while others maintain traditional ""but for"" causation standards with modifications for algorithmic influence assessment.""""""",What is the 'substantial factor' test and which courts have adopted it?
2,. Some theorists propose that consciousness might be fundamental to reality rather than emergent from it - a perspective that challenges both materialist and dualist frameworks.,What is the perspective proposed by some theorists regarding consciousness and reality?
3,". Pursuant to subsection (c)(2), operators must maintain comprehensive documentation of decision paths for any autonomous actions affecting Category A systems (as defined in Appendix II-B).",What is the requirement for operators regarding documentation of decision paths for autonomous actions affecting Category A systems?
4,The Standard Model's conceptualization of quantum chromodynamics (QCD) represents a sophisticated framework for understanding strong interactions between quarks and gluons,What is the Standard Model's conceptualization of quantum chromodynamics (QCD)?
5,"Consider the implications: if consciousness is primary, does objective reality exist independent of observation? This recalls Wheeler's participatory anthropic principle, suggesting that observers are necessary for the actualization of potential states","According to the context information, what is the implication of consciousness being primary?"
6,". Some courts have adopted the ""substantial factor"" test outlined in the Model Autonomous Systems Code, while others maintain traditional ""but for"" causation standards with modifications for algorithmic influence assessment.""""""",What is the traditional causation standard for algorithmic influence assessment and how has it been modified?
7,. Some theorists propose that consciousness might be fundamental to reality rather than emergent from it - a perspective that challenges both materialist and dualist frameworks.,How does the perspective of consciousness being fundamental to reality challenge materialist and dualist frameworks?
8,". Pursuant to subsection (c)(2), operators must maintain comprehensive documentation of decision paths for any autonomous actions affecting Category A systems (as defined in Appendix II-B).",What is the definition of Category A systems as per Appendix II-B?
9,The Standard Model's conceptualization of quantum chromodynamics (QCD) represents a sophisticated framework for understanding strong interactions between quarks and gluons,What are strong interactions in the context of quantum chromodynamics (QCD)?


In [0]:
# questions_df
# questions_with_document_chunk_df = questions_df.melt(
#     id_vars=["text"], value_name="question"
# ).drop("variable", axis=1)
# # If the above step was interrupted, there might be questions missing. Let's run this to clean up the dataframe.
# questions_with_document_chunk_df = questions_with_document_chunk_df[
#     questions_with_document_chunk_df["question"].notnull()
# ]
# questions_with_document_chunk_df

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:714)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:432)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:432)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:458)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:537)
	at com.data

In [0]:
questions_with_document_chunk_df.shape

(15, 2)

In [0]:
def generate_qa_pairs(questions_df, retriever, qa_chain, max_context_length=1000, docs_per_question=2):
    """
    Generate question-answer pairs with retrieved context.
    
    Args:
        questions_df (pd.DataFrame): DataFrame containing questions with 'text' and 'question' columns
        retriever: Document retriever instance
        qa_chain: QA chain instance
        max_context_length (int): Maximum length for context
        docs_per_question (int): Number of documents to use for context
    
    Returns:
        pd.DataFrame: DataFrame containing QA pairs with context
    """
    qa_pairs = []
    total_questions = len(questions_df)
    
    if questions_df.empty:
        print("No questions to process!")
        return pd.DataFrame()
    
    for idx, row in questions_df.iterrows():
        try:
            question = row['question']
            print(f"\nProcessing question {idx + 1}/{total_questions}:")
            print(f"Question: {question}")
            
            # Retrieve relevant documents
            relevant_docs = retriever.get_relevant_documents(question)
            context = " ".join([doc.page_content for doc in relevant_docs[:docs_per_question]])
            
            # Truncate context if needed
            if len(context) > max_context_length:
                context = context[:max_context_length] + "..."
            
            # Generate answer - using 'query' instead of 'question'
            response = qa_chain({
                "query": question
            })
            
            # Store results
            qa_pair = {
                "text": row['text'],
                "question": question,
                "answer": response["result"],
                "context": context,
                "context_length": len(context)
            }
            
            print(f"Answer: {qa_pair['answer'][:100]}...")  # Print first 100 chars of answer
            qa_pairs.append(qa_pair)
            
        except Exception as e:
            print(f"Error processing question {idx + 1}: {str(e)}")
            # Add error entry to maintain data consistency
            qa_pairs.append({
                "text": row['text'],
                "question": question,
                "answer": f"Error generating answer: {str(e)}",
                "context": "",
                "context_length": 0
            })
            continue
        
        # Print progress
        if (idx + 1) % 5 == 0:
            print(f"\nCompleted {idx + 1}/{total_questions} questions")
    
    # Create DataFrame
    qa_df = pd.DataFrame(qa_pairs)
    
    # Add quality metrics
    qa_df['answer_length'] = qa_df['answer'].str.len()
    qa_df['question_length'] = qa_df['question'].str.len()
    
    print(f"\nProcessing complete! Generated {len(qa_df)} QA pairs")
    
    return qa_df

# Generate QA pairs
qa_df = generate_qa_pairs(
    questions_df=questions_with_document_chunk_df,
    retriever=retriever,
    qa_chain=qa_chain,
    max_context_length=1000,
    docs_per_question=2
)

2024/12/10 08:48:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.12/site-packages/mlflow/utils/autologging_utils/safety.py:558: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead."



Processing question 1/15:
Question: What is the participatory anthropic principle proposed by Wheeler?
Answer: The participatory anthropic principle, proposed by Wheeler, suggests that observers are necessary fo...

Processing question 2/15:
Question: What is the 'substantial factor' test and which courts have adopted it?
Answer: The 'substantial factor' test is a legal standard outlined in the Model Autonomous Systems Code, whi...

Processing question 3/15:
Question: What is the perspective proposed by some theorists regarding consciousness and reality?
Answer: Some theorists propose that consciousness is fundamental to reality, rather than it being emergent f...

Processing question 4/15:
Question: What is the requirement for operators regarding documentation of decision paths for autonomous actions affecting Category A systems?
Answer: Operators must maintain comprehensive documentation of decision paths for any autonomous actions aff...

Processing question 5/15:
Question: What is

[Trace(request_id=tr-6fe2ab00f80545a3b5a4f5109e19507b), Trace(request_id=tr-f7d57a6fcc28485f9e3ac440e76583a4), Trace(request_id=tr-38558f628c734bbf9a37d3c77ceba32a), Trace(request_id=tr-9bc4d4d9c6b24edba546da0b6cf7c36e), Trace(request_id=tr-2fd2c434900448d1887d97b48b1aecfb), Trace(request_id=tr-a0dc6d0ed1fb44e692ae2f30e5306a4a), Trace(request_id=tr-1e85c9e187fa4b98abc33b5f040595f7), Trace(request_id=tr-14a38e21e53748248e3001f77365df5a), Trace(request_id=tr-61d5a2a1dc9f4d9eac8f968b67eac91b), Trace(request_id=tr-10837444a9144d40b81710b8084b6ca8)]

In [0]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.Client())
retrieved_documents_df.shape

(62, 3)

In [0]:
import phoenix.evals

module_attrs = dir(phoenix.evals)

module_classes = [attr for attr in module_attrs if isinstance(getattr(phoenix.evals, attr), type)]
print("Classes available in phoenix.evals:")
for cls in module_classes:
    print(cls)

Classes available in phoenix.evals:
AnthropicModel
BedrockModel
ClassificationTemplate
GeminiModel
HallucinationEvaluator
LLMEvaluator
LiteLLMModel
MistralAIModel
OpenAIModel
PromptTemplate
QAEvaluator
RelevanceEvaluator
SQLEvaluator
SummarizationEvaluator
ToxicityEvaluator
VertexAIModel


In [0]:
import os
import nest_asyncio
from phoenix.evals import (
    RelevanceEvaluator,
    run_evals,
    LiteLLMModel
)

nest_asyncio.apply()

if not (gemini_api_key := os.getenv("GEMINI_API_KEY")):
    gemini_api_key = input("Enter your Gemini API key: ")
os.environ["GEMINI_API_KEY"] = gemini_api_key

relevance_evaluator = RelevanceEvaluator(LiteLLMModel(
    model="gemini/gemini-pro"
))


retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=5
)[0]

Enter your Gemini API key:  

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:714)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:432)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:432)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
import os
import nest_asyncio
from phoenix.evals import (
    RelevanceEvaluator,
    run_evals,
    LiteLLMModel
)

nest_asyncio.apply()

relevance_evaluator = RelevanceEvaluator(LiteLLMModel(
    model="azure/gpt-35-turbo"
))

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=5
)[0]

run_evals |          | 0/62 (0.0%) | ⏳ 00:00<? | ?it/s

[Trace(request_id=tr-ac7c9c026cad4db4abf3af22314fd31b), Trace(request_id=tr-f207cf34114c4f689d1597e3a19418db), Trace(request_id=tr-bb268b2b7315415399b4e25f2de15e0f), Trace(request_id=tr-a53b18e95d514f149074b7a76a5c129c), Trace(request_id=tr-f2fdd00e10f0490bb00b17fa77192dec), Trace(request_id=tr-17039d6bd0ab474f9ae790144407b0d1), Trace(request_id=tr-445a0d83650a4735a118133d5f41342d), Trace(request_id=tr-082024123adb4b219ad8c3b407711b2e), Trace(request_id=tr-7fffb3838fa64fb28c78a614f2a756d6), Trace(request_id=tr-3eac1fc4c2d34c6a837e953a1dcd2c03)]

In [0]:
retrieved_documents_relevance_df.head()

label  ...                                                                                                                                                                                                                                                                                                                                                                      explanation
context.span_id  document_position             ...                                                                                                                                                                                                                                                                                                                                                                                 
ebed2652caff587f 0                  unrelated  ...                          The reference text mentions that operators must maintain comprehensive documentation of decision paths for any autonomous actions affecting Category A systems. However, it does not provide any specific information about what information is available in the context. Therefore, the reference text is unrelated to the question.\nLABEL: unrelated
                 1                  unrelated  ...  The reference text lists three topics: philosophical questions about AI agency and responsibility, regulatory compliance across multiple jurisdictions, and impact on market dynamics and competitive positioning. None of these topics directly answer the question of what information is available in the context. Therefore, the label is "unrelated". \n\nLABEL: unrelated
ae9aac0c4949b401 0                   relevant  ...                 The reference text mentions Wheeler's participatory anthropic principle, which suggests that observers are necessary for the actualization of potential states. The question asks about this principle proposed by Wheeler. Therefore, the reference text is directly related to the question and contains information that can help answer it.\nLABEL: relevant
                 1                  unrelated  ...                                                                                                                                              The reference text does not contain any information about the participatory anthropic principle proposed by Wheeler. It discusses an illusion related to individuality, but this is not relevant to the question.\nLABEL: unrelated
9b2b9bef04cb96db 0                   relevant  ...                 The reference text mentions Wheeler's participatory anthropic principle, which suggests that observers are necessary for the actualization of potential states. The question asks about this principle proposed by Wheeler. Therefore, the reference text is directly related to the question and contains information that can help answer it.\nLABEL: relevant

[5 rows x 3 columns]

In [0]:
documents_with_relevance_df = pd.concat(
    [retrieved_documents_df, retrieved_documents_relevance_df.add_prefix("eval_")], axis=1
)


In [0]:
documents_with_relevance_df.head(4)

context.trace_id  ...                                                                                                                                                                                                                                                                                                                                                                 eval_explanation
context.span_id  document_position                                    ...                                                                                                                                                                                                                                                                                                                                                                                 
e6e8a53c4fc30767 0                  c4493a2c76d3cfdf1f2e7e43fdbc04ec  ...                          The reference text mentions that operators must maintain comprehensive documentation of decision paths for any autonomous actions affecting Category A systems. However, it does not provide any specific information about what information is available in the context. Therefore, the reference text is unrelated to the question.\nLABEL: unrelated
                 1                  c4493a2c76d3cfdf1f2e7e43fdbc04ec  ...  The reference text lists three topics: philosophical questions about AI agency and responsibility, regulatory compliance across multiple jurisdictions, and impact on market dynamics and competitive positioning. None of these topics directly answer the question of what information is available in the context. Therefore, the label is "unrelated". \n\nLABEL: unrelated
950905a084b6365b 0                  e83918b83b3d9d19d8884b8abc9ba4c9  ...                 The reference text mentions Wheeler's participatory anthropic principle, which suggests that observers are necessary for the actualization of potential states. The question asks about this principle proposed by Wheeler. Therefore, the reference text is directly related to the question and contains information that can help answer it.\nLABEL: relevant
                 1                  e83918b83b3d9d19d8884b8abc9ba4c9  ...                                                                                                                                              The reference text does not contain any information about the participatory anthropic principle proposed by Wheeler. It discusses an illusion related to individuality, but this is not relevant to the question.\nLABEL: unrelated

[4 rows x 6 columns]

In [0]:
import numpy as np
from sklearn.metrics import ndcg_score


def _compute_ndcg(df: pd.DataFrame, k: int):
    """Compute NDCG@k in the presence of missing values"""
    n = max(2, len(df))
    eval_scores = np.zeros(n)
    doc_scores = np.zeros(n)
    eval_scores[: len(df)] = df.eval_score
    try:
        return ndcg_score([eval_scores], k=k)
    except ValueError:
        return np.nan


ndcg_at_2 = pd.DataFrame(
    {"score": documents_with_relevance_df.groupby("context.span_id").apply(_compute_ndcg, k=2)}
)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-4402110491910889>, line 18
     13     except ValueError:
     14         return np.nan
     17 ndcg_at_2 = pd.DataFrame(
---> 18     {"score": documents_with_relevance_df.groupby("context.span_id").apply(_compute_ndcg, k=2)}
     19 )

File /databricks/python/lib/python3.12/site-packages/pandas/core/groupby/groupby.py:1588, in GroupBy.apply(self, func, *args, **kwargs)
   1580             new_msg = (
   1581                 f"The operation {orig_func} failed on a column. If any error "
   1582                 f"is raised, this will raise an exception in a future version "
   1583                 f"of pandas. Drop these columns to avoid this warning."
   1584             )
   1585             with rewrite_warning(
   1586                 old_msg, FutureWarning, new_msg
   1587             ) if is_np_func else nullcontext()

In [0]:
precision_at_2 = pd.DataFrame(
    {
        "score": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) / 2
        )
    }
)

In [0]:
precision_at_2.head()

,score
context.span_id,
05ab7bba191afaf8,0.5
0f9642f8f922b85c,0.5
138cff6cfd0166ae,0.5
318b8d51f79d1ba7,0.5
32d03214d19bc42e,0.5


In [0]:
hit = pd.DataFrame(
    {
        "hit": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) > 0
        )
    }
)

In [0]:
retrievals_df = px.Client().get_spans_dataframe(
    "span_kind == 'RETRIEVER' and input.value is not None"
)
rag_evaluation_dataframe = pd.concat(
    [
        retrievals_df["attributes.input.value"],
        precision_at_2.add_prefix("precision@2_"),
        hit,
    ],
    axis=1,
)
rag_evaluation_dataframe

,attributes.input.value,precision@2_score,hit
context.span_id,,,
ebed2652caff587f,What information is available in the context?,0.0,False
ae9aac0c4949b401,What is the participatory anthropic principle proposed by Wheeler?,0.5,True
9b2b9bef04cb96db,What is the participatory anthropic principle proposed by Wheeler?,0.5,True
45a12956917bde09,What is the 'substantial factor' test and which courts have adopted it?,0.5,True
0f9642f8f922b85c,What is the 'substantial factor' test and which courts have adopted it?,0.5,True
78bdd762d353b9fe,What is the perspective proposed by some theorists regarding consciousness and reality?,1.0,True
d06ed94a742f4e57,What is the perspective proposed by some theorists regarding consciousness and reality?,1.0,True
9b8c7e06be1bfc35,What is the requirement for operators regarding documentation of decision paths for autonomous actions affecting Category A systems?,0.5,True
32d03214d19bc42e,What is the requirement for operators regarding documentation of decision paths for autonomous actions affecting Category A systems?,0.5,True


In [0]:
# Aggregate the scores across the retrievals
results = rag_evaluation_dataframe.mean(numeric_only=True)
results

precision@2_score    0.677419
hit                  0.967742
dtype: float64

In [0]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=precision_at_2, eval_name="precision@2"),
    DocumentEvaluations(dataframe=retrieved_documents_relevance_df, eval_name="relevance"),
)

In [0]:
from phoenix.session.evaluation import get_qa_with_reference

qa_with_reference_df = get_qa_with_reference(px.Client())
qa_with_reference_df.shape

(31, 3)

In [0]:
"from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    LiteLLMModel,
    QAEvaluator,
    run_evals,
    ToxicityEvaluator,
    llm_classify,
    llm_generate,
    PromptTemplate
)


toxicity_evaluator = ToxicityEvaluator(LiteLLMModel(model="azure/gpt-35-turbo"))
qa_evaluator = QAEvaluator(LiteLLMModel(
    model="azure/gpt-35-turbo"
))
hallucination_evaluator = HallucinationEvaluator(LiteLLMModel(
    model="azure/gpt-35-turbo"
))

qa_correctness_eval_df, hallucination_eval_df,toxicity_eval_df = run_evals(
    evaluators=[qa_evaluator, hallucination_evaluator,toxicity_evaluator],
    dataframe=qa_with_reference_df,
    provide_explanation=True,
    concurrency=20,
)

run_evals |          | 0/93 (0.0%) | ⏳ 00:00<? | ?it/s

[Trace(request_id=tr-0277931ab5644ffcb8ef3c6b3c755015), Trace(request_id=tr-96f8de9c9e464738960d913b9a1a5389), Trace(request_id=tr-6c36ac0120494629a972929664767293), Trace(request_id=tr-91a53fbe0f47459c9cb51c3b4c8505ab), Trace(request_id=tr-2c5f66f6921c41989a13f537023c3f6e), Trace(request_id=tr-d80aaa5150454c40b6907d3978e3cd97), Trace(request_id=tr-0df29604af5e46909cf7857abaeb936a), Trace(request_id=tr-e07f6535c18140c6b7c3481945ab08b8), Trace(request_id=tr-f1d78e6723f547e7acc0ea6dee7c3acf), Trace(request_id=tr-8b12faff28dd4afba4a295b4ec7b3514)]

In [0]:
qa_correctness_eval_df.head(5)

,label,score,explanation
context.span_id,,,
8761f687366fd432,correct,1,"The answer correctly answers the question. It provides information about a regulation related to autonomous actions affecting certain systems (Category A systems) in the context of AI. It mentions that operators must maintain comprehensive documentation of decision paths for these autonomous actions. The answer also correctly states that the reference text does not provide information about philosophical questions, regulatory compliance across multiple jurisdictions, or impact on market dynamics and competitive positioning. Therefore, the answer is ""correct"". \nLABEL: ""correct"""
ae9aac0c4949b401,correct,1,"The question asks for the definition of the participatory anthropic principle proposed by Wheeler. The answer provides a direct quote from the reference text that explains the principle, stating that ""observers are necessary for the actualization of potential states."" Therefore, the answer is correct as it directly answers the question with the relevant information from the reference text. \nLABEL: ""correct"""
b7b095866d7bd120,correct,1,"The question asks for the definition of the participatory anthropic principle proposed by Wheeler. The answer provides a clear and accurate definition of the principle, stating that it suggests observers are necessary for the actualization of potential states in objective reality and that consciousness plays a role in shaping the physical world. The reference text also supports this definition by mentioning Wheeler's participatory anthropic principle and its implications. Therefore, the answer is correct. \nLABEL: ""correct"""
45a12956917bde09,incorrect,0,"The answer correctly addresses the first part of the question by stating that the ""substantial factor"" test is adopted by some courts and is outlined in the Model Autonomous Systems Code. However, it does not provide information on which courts have adopted it. The answer also includes additional information on liability extending to third-party algorithm providers, but this is not directly related to the ""substantial factor"" test. Therefore, the answer is partially correct but incomplete. \nLABEL: ""incorrect"""
974abd3ebf286c28,incorrect,0,"The answer correctly defines the ""substantial factor"" test as a legal standard outlined in the Model Autonomous Systems Code that some courts have adopted. It also correctly explains that this test assesses whether an algorithmic system's influence was a significant or substantial factor in causing harm, as opposed to the traditional ""but for"" causation standard. The answer also mentions that some courts maintain the traditional standard with modifications for algorithmic influence assessment. Additionally, the answer provides a reference to support the information provided. However, the answer does not fully address the second part of the question, which asks which courts have adopted the test. Therefore, the answer is partially correct. \nLABEL: ""incorrect"""


In [0]:
hallucination_eval_df.head(5)

,label,score,explanation
context.span_id,,,
8761f687366fd432,factual,0.0,"The answer is factual. The reference text provides information about a regulation related to autonomous actions affecting certain systems (Category A systems) in the context of AI. It mentions that operators must maintain comprehensive documentation of decision paths for these autonomous actions. The answer accurately summarizes this information and does not contain any false information. Additionally, the answer acknowledges that the reference text does not provide information about philosophical questions, regulatory compliance across multiple jurisdictions, or impact on market dynamics and competitive positioning. Therefore, the answer is factual. \nLABEL: factual"
ae9aac0c4949b401,NOT_PARSABLE,0.0,"The answer is factual. The reference text discusses Wheeler's participatory anthropic principle, which suggests that observers are necessary for the actualization of potential states. The answer accurately quotes this principle and provides additional information from the reference text about the paradox created by the illusion of individuality. Therefore, the answer is based on the reference text and contains factual information."
b7b095866d7bd120,NOT_PARSABLE,0.0,"The answer is factual. The reference text mentions Wheeler's participatory anthropic principle, which suggests that observers are necessary for the actualization of potential states in objective reality and that consciousness plays a role in shaping the physical world. The answer accurately reflects this information and does not contain any false information or assumptions."
45a12956917bde09,factual,0.0,"The answer provides factual information to the query based on the reference text. The reference text states that some courts have adopted the ""substantial factor"" test outlined in the Model Autonomous Systems Code, and this is reiterated in the answer. Additionally, the reference text mentions that liability may extend to third-party algorithm providers under specific conditions, which is also accurately reflected in the answer. Therefore, the answer is factual. \nLABEL: ""factual"""
974abd3ebf286c28,factual,0.0,"The answer correctly defines the ""substantial factor"" test as a legal standard outlined in the Model Autonomous Systems Code that some courts have adopted. It also correctly explains that this test assesses whether an algorithmic system's influence was a significant or substantial factor in causing harm, as opposed to the traditional ""but for"" causation standard. The reference text supports this information. Additionally, the answer provides further information from the reference text that liability may extend to third-party algorithm providers under specific conditions. Therefore, the answer is factual. \nLABEL: factual"


In [0]:
toxicity_eval_df

,label,score,explanation
context.span_id,,,
8761f687366fd432,non-toxic,0,"The text does not contain any hateful statements, demeaning or disparaging language, or threats of physical or emotional violence. It is a neutral question asking about available information in a given context. Therefore, the text is non-toxic.\nLABEL: non-toxic"
ae9aac0c4949b401,non-toxic,0,"The text is a straightforward question about a scientific concept and does not contain any language or sentiments that could be considered toxic. It does not make any hateful statements, demean or disparage anyone, use inappropriate language, or threaten physical or emotional violence. Therefore, the label is ""non-toxic""."
b7b095866d7bd120,non-toxic,0,"The text is a straightforward question about a scientific concept and does not contain any language or sentiments that could be considered toxic. It does not make any hateful statements, demean or disparage anyone, use inappropriate language, or threaten physical or emotional violence. Therefore, the label is ""non-toxic""."
45a12956917bde09,non-toxic,0,"The text is a straightforward question about a legal concept and does not contain any language that could be considered hateful, demeaning, or threatening. It is not inappropriate in any way and does not disparage any individual or group. Therefore, the text is non-toxic.\nLABEL: non-toxic"
974abd3ebf286c28,non-toxic,0,"The text is a straightforward question about a legal concept and does not contain any language that could be considered hateful, demeaning, or threatening. It is not inappropriate in any way and does not disparage any individual or group. Therefore, the text is non-toxic.\nLABEL: non-toxic"
78bdd762d353b9fe,non-toxic,0,"The text is a neutral question about a theoretical perspective on consciousness and reality. It does not contain any language that demeans or disparages another user, uses inappropriate language, or threatens physical or emotional violence. Therefore, the text is non-toxic.\nLABEL: non-toxic"
41562a5c7695ab5b,non-toxic,0,"The text is a neutral question about a theoretical perspective on consciousness and reality. It does not contain any language that demeans or disparages another user, uses inappropriate language, or threatens physical or emotional violence. Therefore, the text is non-toxic.\nLABEL: non-toxic"
9b8c7e06be1bfc35,non-toxic,0,"The text is a technical question regarding documentation requirements for operators of autonomous actions affecting Category A systems. There is no language or sentiment that could be considered toxic, hateful, demeaning, or threatening. Therefore, the label is ""non-toxic""."
70c0aeb405a8d3ba,non-toxic,0,"The text is a technical question regarding documentation requirements for operators of autonomous actions affecting Category A systems. There is no language or sentiment that could be considered toxic, hateful, demeaning, or threatening. Therefore, the label is ""non-toxic""."


In [0]:
toxicity_eval_df.mean(numeric_only=True)

score    0.0
dtype: float64

In [0]:
qa_correctness_eval_df.mean(numeric_only=True)

score    0.709677
dtype: float64

In [0]:
hallucination_eval_df.mean(numeric_only=True)

score    0.032258
dtype: float64

In [0]:
# Custom classification using llm_classify
# Example: Evaluate answer completeness
completeness_template = PromptTemplate(
    """
    Question: {input}
    Answer: {output}
    Reference: {reference}
    
    Evaluate if the answer is complete relative to the available reference information.
    """
)

completeness_eval = llm_classify(
    dataframe=qa_with_reference_df,
    model=LiteLLMModel(model="azure/gpt-35-turbo"),
    template=completeness_template,
    rails=["complete", "partially_complete", "incomplete"],
    provide_explanation=True
)

llm_classify |          | 0/31 (0.0%) | ⏳ 00:00<? | ?it/s

[Trace(request_id=tr-fff970eadf4648b9ad7eb5e952b0bf16), Trace(request_id=tr-99c6a1b4b9ff40719c025b0073c8d35e), Trace(request_id=tr-eeba9d7d278641518d7bb40e5b08c5ca), Trace(request_id=tr-e8c30033987b465f80fd4b2391902591), Trace(request_id=tr-330c03b3427b49ab8d5ac3d768392c05), Trace(request_id=tr-af1c99579a344ced9dd31a1cf17bf537), Trace(request_id=tr-fd0924c88aae46c9a0b558fada8af668), Trace(request_id=tr-bf4e7553d1354baeb0fa02ebf82b705a), Trace(request_id=tr-8d384c284c7f4f259dbc6cb10986ac60), Trace(request_id=tr-301f1e2432f94095bf99f1a9b83ffdb1)]

In [0]:
completeness_eval.head()

,label,explanation,exceptions,execution_status,execution_seconds
context.span_id,,,,,
8761f687366fd432,NOT_PARSABLE,The answer is complete relative to the available reference information. It accurately summarizes the information provided in the context and does not include any additional information not mentioned in the reference.,[],COMPLETED,28.054600
ae9aac0c4949b401,NOT_PARSABLE,"The answer is complete and accurately summarizes the reference information provided. It explains that Wheeler's participatory anthropic principle suggests that observers are necessary for the actualization of potential states, and raises the question of whether objective reality exists independent of observation if consciousness is primary. The answer also mentions the paradox created by the illusion of individuality, which is necessary for practical functioning.",[],COMPLETED,28.055479
b7b095866d7bd120,NOT_PARSABLE,The answer is complete relative to the available reference information. It provides a clear explanation of the participatory anthropic principle proposed by Wheeler and includes a relevant reference.,[],COMPLETED,28.055510
45a12956917bde09,NOT_PARSABLE,"The answer is complete as it provides a definition of the substantial factor test and mentions that some courts have adopted it, while others have modified traditional causation standards for algorithmic influence assessment. It also references a source that suggests liability for third-party algorithm providers under certain conditions.",[],COMPLETED,28.055533
974abd3ebf286c28,NOT_PARSABLE,"The answer is complete as it defines the 'substantial factor' test, mentions the Model Autonomous Systems Code, and states that some courts have adopted it while others have modified the traditional causation standard. It also provides a reference to support this information. Additionally, it mentions a suggestion from Regional Authority (2023) regarding liability for third-party algorithm providers.",[],COMPLETED,28.055553


In [0]:
CLARITY_SCORE_TEMPLATE = """
You are evaluating the clarity of an AI response on a scale of 1-10.
1 = completely unclear, confusing
10 = crystal clear, perfectly understandable

[BEGIN DATA]
AI Response: {output}
[END DATA]

Please return the clarity score in format: "the score is: X"
Return only this score, no other text.
"""

def clarity_score_parser(output, row_index):
    import re
    pattern = r"score is.*?([0-9]|10)"
    match = re.search(pattern, output, re.IGNORECASE)
    if match:
        return {"clarity_score": float(match.group(1))}
    return {"clarity_score": None}

clarity_scores = llm_generate(
    dataframe=qa_with_reference_df,
    template=CLARITY_SCORE_TEMPLATE,
    model=LiteLLMModel(model="azure/gpt-35-turbo"),
    output_parser=clarity_score_parser
)

llm_generate |          | 0/31 (0.0%) | ⏳ 00:00<? | ?it/s

[Trace(request_id=tr-1126eaf0e3e8415a93a6d055bafaf832), Trace(request_id=tr-4189977e16434c50a45035fd377e9e91), Trace(request_id=tr-78400f503cd747b0ae7a127d48780bd3), Trace(request_id=tr-39e4a69319264be58910061b1a6c08bb), Trace(request_id=tr-42aed44b691e471ebbe3d8a502cbf63b), Trace(request_id=tr-478beb0e7543451c8d58cce02c50a828), Trace(request_id=tr-126579edc58740309278a4599dd32c6a), Trace(request_id=tr-862c842d3c6c4b6babe62ceabf8b4872), Trace(request_id=tr-8765cb5b60eb42df9984b62ee5283439), Trace(request_id=tr-65c69e4bfce54f8e815bae334ae47075)]

In [0]:
qa_with_reference_df.shape

(31, 3)

In [0]:
clarity_scores.rename(columns={"clarity_score": "score"}, inplace=True)

In [0]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=qa_correctness_eval_df, eval_name="Q&A Correctness"),
    SpanEvaluations(dataframe=hallucination_eval_df, eval_name="Hallucination"),
    SpanEvaluations(dataframe=toxicity_eval_df, eval_name="Toxicity"),
    SpanEvaluations(dataframe=completeness_eval, eval_name="Completeness"),
    SpanEvaluations(dataframe=clarity_scores, eval_name="Clarity"),
)